In [1]:
import spiketag.fpga.memory_api as mapi

In [2]:
!hexdump -C -v -n 128 /dev/xillybus_mem_16

00000000  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000010  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000020  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000030  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000040  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000050  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000060  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000070  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000080


### w/r mem_16 from strach

In [3]:
import numpy as np
import struct
from binascii import hexlify
import io

In [4]:
def to_fixed_point(v, dtype='<i4', binpoint=14):
    v = int(v * 2**binpoint)
    value = hexlify(struct.pack(dtype,v))
    # print value
    return struct.pack(dtype,v)

def to_value(hexstring, dtype='<i4', binpoint=14):
    value = struct.unpack(dtype, hexstring)
    value = float(value[0]) / 2**binpoint
    return value

Test this part of verilog:


```verilog

if (user_r_control_regs_16_rden)
    begin
        case(user_control_regs_16_addr)
            5'h00: user_r_control_regs_16_data <= {13'b0, DSP_settle, SPI_run_continuous_in, 1'b0};
            5'h01: user_r_control_regs_16_data <= max_timestep_in[15:0];
            5'h02: user_r_control_regs_16_data <= max_timestep_in[31:16];
            5'h03: user_r_control_regs_16_data <= {dataclk_D[3], dataclk_M, dataclk_O};
            5'h04: user_r_control_regs_16_data <= {delay_D, delay_C, delay_B, delay_A};
            5'h08: user_r_control_regs_16_data <= {aux_cmd_bank_1_D_in, aux_cmd_bank_1_C_in, aux_cmd_bank_1_B_in, aux_cmd_bank_1_A_in};
            5'h09: user_r_control_regs_16_data <= {aux_cmd_bank_2_D_in, aux_cmd_bank_2_C_in, aux_cmd_bank_2_B_in, aux_cmd_bank_2_A_in};
            5'h0A: user_r_control_regs_16_data <= {aux_cmd_bank_3_D_in, aux_cmd_bank_3_C_in, aux_cmd_bank_3_B_in, aux_cmd_bank_3_A_in};
            5'h0B: user_r_control_regs_16_data <= {6'b0, max_aux_cmd_index_1_in};
            5'h0C: user_r_control_regs_16_data <= {6'b0, max_aux_cmd_index_2_in};
            5'h0D: user_r_control_regs_16_data <= {6'b0, max_aux_cmd_index_3_in};
            5'h0E: user_r_control_regs_16_data <= {6'b0, loop_aux_cmd_index_1};
            5'h0F: user_r_control_regs_16_data <= {6'b0, loop_aux_cmd_index_2};
            5'h10: user_r_control_regs_16_data <= {6'b0, loop_aux_cmd_index_3};
            5'h12: user_r_control_regs_16_data <= {data_stream_4_sel, data_stream_3_sel,  data_stream_2_sel,  data_stream_1_sel};  
            5'h13: user_r_control_regs_16_data <= {data_stream_8_sel, data_stream_7_sel,  data_stream_6_sel,  data_stream_5_sel};
            5'h14: user_r_control_regs_16_data <= {data_stream_12_sel, data_stream_11_sel,  data_stream_10_sel,  data_stream_9_sel};
            5'h15: user_r_control_regs_16_data <= {data_stream_16_sel, data_stream_15_sel,  data_stream_14_sel,  data_stream_13_sel};
            5'h16: user_r_control_regs_16_data <= {
            data_stream_16_en_in,
            data_stream_15_en_in,
            data_stream_14_en_in,
            data_stream_13_en_in,
            data_stream_12_en_in,
            data_stream_11_en_in,
            data_stream_10_en_in,
            data_stream_9_en_in,
            data_stream_8_en_in,
            data_stream_7_en_in,
            data_stream_6_en_in,
            data_stream_5_en_in,
            data_stream_4_en_in,
            data_stream_3_en_in,
            data_stream_2_en_in,
            data_stream_1_en_in
            };
            5'h17: user_r_control_regs_16_data <= aux_output;
            default:  user_r_control_regs_16_data <= 16'b0;
        endcase
    end
end

```


*open bidirectional memory as mode `rb+`, you should see reset led off*

In [5]:
reg16 = open('/dev/xillybus_mem_16', 'rb+')

*There are 32 address for `xillybus_control_regs_16`*

In [8]:
addrs = np.arange(0,63,2).astype(int)
value = to_fixed_point(78, '<h', 0)

*write the registers, you should see long led pattern*

In [9]:
i = 2
reg16.seek(addrs[i])
reg16.write(value)
reg16.flush()

*read the registers, you should see short led pattern*

In [10]:
reg16.seek(addrs[i])
hexstring = reg16.read(2)
hexstring

'N\x00'

In [11]:
return_val = to_value(value, '<h', 0)
print return_val

78.0


*close device, you should see reset led on*

In [12]:
reg16.close()

### w/r mem_16 from api

In [7]:
from spiketag.fpga.memory_api import read_mem_16, write_mem_16, mem_reset

In [13]:
write_mem_16(0,1)

IOError: [Errno 16] Device or resource busy: '/dev/xillybus_mem_16'

In [14]:
read_mem_16(0)

IOError: [Errno 16] Device or resource busy: '/dev/xillybus_mem_16'

In [11]:
!hexdump -C -v -n 64 /dev/xillybus_mem_16

hexdump: /dev/xillybus_mem_16: Device or resource busy
